# DX 704 Week 3 Project

This week's project will give you practice with optimizing choices for bandit algorithms.
You will be given access to the bandit problem via a blackbox object, and you will investigate the bandit rewards to pick a suitable algorithm.

The full project description, a template notebook and supporting code are available on GitHub: [Project 3 Materials](https://github.com/bu-cds-dx704/dx704-project-03).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Pick a Bandit Algorithm

Experiment with the multi-armed bandit interface using seed 0 to learn about the distribution of rewards and decide what kind of bandit algorithm will be appropriate.
A histogram will likely be helpful.

In [41]:
# DO NOT CHANGE

import numpy as np

class BanditProblem(object):
    def __init__(self, seed):
        self.seed = seed
        self.rng = np.random.default_rng(seed)

        self.num_arms = 3
        self.ns = self.rng.integers(low=1, high=10, size=self.num_arms)
        self.ps = self.rng.uniform(low=0.2, high=0.4, size=self.num_arms)

    def get_num_arms(self):
        return self.num_arms

    def get_reward(self, arm):
        if arm < 0 or arm >= self.num_arms:
            raise ValueError("Invalid arm")

        x = self.rng.uniform()
        x *= self.rng.binomial(self.ns[arm], self.ps[arm])

        return x


In [42]:
bandit0 = BanditProblem(0)

In [43]:
bandit0.get_num_arms()

3

In [44]:
bandit0.get_reward(arm=0)

1.8255111545554434

In [45]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd
from math import log, sqrt

In [48]:
# Upper Confidence Bound

def run_ucb(bandit, T=3000, R_max=9.0):
    num_arms = bandit.get_num_arms()
    counts = np.zeros(num_arms, dtype=int)
    sums = np.zeros(num_arms, dtype=float)

    for a in range(num_arms):
        r = bandit.get_reward(a) / R_max
        counts[a] += 1
        sums[a] += r

    total = counts.sum()
    while total < T:
        means = sums / np.maximum(counts, 1)
        bonuses = np.sqrt(2.0 * np.log(total + 1) / counts)
        a = int(np.argmax(means + bonuses))
        r = bandit.get_reward(a) / R_max
        counts[a] += 1
        sums[a] += r
        total += 1

    avg_reward = (sums.sum() / T) * R_max   
    return avg_reward, counts

In [49]:
# Thomspon sampling
def run_thompson_sampling(bandit, T=3000):
    num_arms = bandit.get_num_arms()
    container = [[] for _ in range(K)]
    counts = [0]*num_arms

    for a in range(num_arms):
        r = bandit.get_reward(a)
        container[a].append(r); counts[a] += 1

    for _ in range(sum(counts), T):
        boot_means = [np.mean(np.random.choice(container[a], size=len(container[a]), replace=True)) for a in range(K)]
        a = int(np.argmax(boot_means))
        container[a].append(bandit.get_reward(a)); counts[a] += 1

    avg_reward = np.mean([x for reward_a in container for x in reward_a])
    return avg_reward, np.array(counts), np.array([np.mean(reward_a) for reward_a in container])

In [50]:
# Epsilon greedy

def run_epsilon_greedy(bandit, T=3000, eps0=0.2, decay=0.997):
    num_arms = bandit.get_num_arms()
    counts = np.zeros(K, dtype=int)
    sums = np.zeros(K, dtype=float)
    eps = eps0

    for a in range(num_arms):
        r = bandit.get_reward(a)
        counts[a] += 1
        sums[a] += r

    total = counts.sum()
    while total < T:
        if np.random.random() < eps:
            a = np.random.randint(num_arms)
        else:
            a = int(np.argmax(sums / np.maximum(counts, 1)))
        r = bandit.get_reward(a)
        counts[a] += 1
        sums[a] += r
        total += 1
        eps *= decay

    avg_reward = sums.sum() / T
    return avg_reward, counts

In [51]:
# Result evaluation 

def eval_results(runs=20, T=3000):

    ucb_avgs, eg_avgs, ts_avgs = [], [], []

    for _ in range(runs):
        ucb_avgs.append(run_ucb(BanditProblem(0), T=T, R_max=9.0)[0])
        eg_avgs.append(run_epsilon_greedy(BanditProblem(0), T=T)[0])
        ts_avgs.append(run_thompson_sampling(BanditProblem(0), T=T)[0])

    print(f"UCB   mean={np.mean(ucb_avgs):.4f} ± {np.std(ucb_avgs):.4f}")
    print(f"e-greedy mean={np.mean(eg_avgs):.4f} ± {np.std(eg_avgs):.4f}")
    print(f"TS mean={np.mean(ts_avgs):.4f} ± {np.std(ts_avgs):.4f}")

In [52]:
eval_results(20, 3000)

UCB   mean=0.7994 ± 0.0000
e-greedy mean=0.8849 ± 0.0168
TS mean=0.8804 ± 0.0077


Based on your investigation, pick an appropriate bandit algorithm to implement from the algorithms covered this week.
Write a file "algorithm-choice.txt" that states your choice and gives a few sentences justifying your choice and rejecting the alternatives.
Keep your explanation concise; overly verbose responses will be penalized.

## Part 2: Implement Bandit

Based on your decision, implement an appropriate bandit algorithm and pick 1000 actions using seed 2025002.

In [53]:
# YOUR CHANGES HERE

def thompson_sampling(env_seed=2025002, T=1000, algo_seed=2025002):

    bandit = BanditProblem(env_seed)
    rng = np.random.default_rng(algo_seed)

    num_arms = bandit.get_num_arms()
    container = [[] for _ in range(num_arms)]
    actions, rewards = [], []

    for a in range(num_arms):
        r = bandit.get_reward(a)
        container[a].append(r)
        actions.append(a)
        rewards.append(r)

    for _ in range(num_arms, T):
        boot_means = []
        for a in range(num_arms):
            reward_a = np.asarray(container[a], dtype=float)
            idx = rng.integers(0, len(reward_a), size=len(reward_a)) 
            boot_means.append(reward_a[idx].mean())
        a_star = int(np.argmax(boot_means))

        r = bandit.get_reward(a_star)
        container[a_star].append(r)
        actions.append(a_star)
        rewards.append(r)

    counts = np.array([len(container[a]) for a in range(K)])
    avg_reward = float(np.mean(rewards))
    return actions, rewards, counts, avg_reward

In [54]:
actions, rewards, counts, avg_reward = thompson_sampling(env_seed=2025002, T=1000, algo_seed=2025002)

In [55]:
#print(actions, rewards, counts, avg_reward)

Write a file "history.tsv" with columns action and reward in the order that the actions were taken.

In [56]:
# YOUR CHANGES HERE

history = pd.DataFrame({"action": actions, "reward": rewards}).to_csv("history.tsv", sep="\t", index=False)

Submit "history.tsv" in Gradescope.

## Part 3: Action Statistics

Based on the data from part 2, estimate the expected reward for each arm and write a file "actions.tsv" with the columns action, min_reward, mean_reward, max_reward.

In [57]:
# YOUR CHANGES HERE

hist = pd.read_csv("history.tsv", sep="\t")

stats = (
    hist.groupby("action", as_index=False)["reward"]
        .agg(min_reward="min", mean_reward="mean", max_reward="max")
        .sort_values("action")
)

In [58]:
stats.to_csv("actions.tsv", sep="\t", index=False)

Submit "actions.tsv" in Gradescope.

## Part 4: Regret Estimates

Calculate the expected regret taking 1000 actions with the following strategies.

* uniform: Pick an arm uniformly at random.
* just-i: Always pick arm $i$. Do this for $i=0$ to $K-1$ where $K$ is the number of arms.
* actual: This should match your output in part 2.

In [59]:
# YOUR CHANGES HERE

env = BanditProblem(2025002)
num_arms = env.get_num_arms()
mu = 0.5 * env.ns * env.ps
mu_star = float(mu.max())

In [60]:
hist = pd.read_csv("history.tsv", sep="\t")
len_hist = len(hist)
counts = hist["action"].value_counts().reindex(range(num_arms), fill_value=0).to_numpy()

In [61]:
rows = [
    ("uniform", len_hist * (mu_star - float(mu.mean()))),
    *[(f"just-{i}", len_hist * (mu_star - float(mu[i]))) for i in range(K)],
    ("actual", len_hist * mu_star - float(np.dot(counts, mu))),
]

Write your results to a file "strategies.tsv" with the columns strategy and regret.

In [62]:
# YOUR CHANGES HERE

strat = pd.DataFrame(rows, columns=["strategy", "regret"]).to_csv("strategies.tsv", sep="\t", index=False)

Submit "strategies.tsv" in Gradescope.

## Part 5: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

Submit "project.ipynb" in Gradescope.